In [1]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,005 kB]
Hit:14 http://ppa.la

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-11 17:52:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.92MB/s    in 0.2s    

2022-10-11 17:52:39 (5.92 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://fvawsbucket.s3.us-west-1.amazonaws.com/amazon_reviews.tsv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews.tsv"), sep='\t', header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [30]:
# Total Votes
total_votes_20_df = df.filter("total_votes>=20")
total_votes_20_count = total_votes_20_df.count()
total_votes_20_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [6]:
# Percentual  - Helpful votes
percentage_votes_df = total_votes_20_df.filter("helpful_votes/total_votes >= 0.5")
percentage_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [7]:
# Percentual  - Helpful votes (PAID)
vine_y_df = percentage_votes_df.filter("vine == 'Y'")
vine_y_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34746219| RG70K1HMY4LXX|B010H76PTU|     581545062|Omron 7 Series Wi...|Health & Personal...|          4|           37|         44|   Y|                N|Only tracks 1 per...|This product come...| 2015-08-27|
|         US|   51401810|R1DHGTNXDXJ0GB|B00VAPRSP4|     373707856|Cellucor, SuperHD...|Health & Personal

In [8]:
# Percentual  - Helpful votes (UNPAID)
vine_n_df = percentage_votes_df.filter("vine == 'N'")
vine_n_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [31]:
# Number of 5-star reviews
five_star_df = total_votes_20_df.filter("star_rating == 5")
five_star_count = five_star_df.count()

In [43]:
# Percentage of 5-star reviews for the two types of review (paid vs unpaid)
five_star_paid_df = five_star_df.filter("vine == 'Y'")
five_star_unpaid_df = five_star_df.filter("vine == 'N'")

five_star_percentage_paid = ((five_star_paid_df.count()/five_star_count)*100)
five_star_percentage_unpaid = ((five_star_unpaid_df.count()/five_star_count) *100)

In [44]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , FloatType

# Total number of reviews (discarding total_votes less than 20) / Number of 5-star reviews / Percentage of 5-star reviews for paid and unpaid
row = [(total_votes_20_count,five_star_count,five_star_percentage_paid,five_star_percentage_unpaid)]

schema = [ \
    StructField("Total number of reviews", IntegerType(), True), \
    StructField("Number of 5-star reviews", IntegerType(), True), \
    StructField("Percentage of 5-star reviews (Paid) - %", FloatType(), True), \
    StructField("Percentage of 5-star reviews (Unpaid) - %", FloatType(), True), \
  ]
final = StructType(fields=schema)


analysis_df = spark.createDataFrame(data=row, schema=final)
analysis_df.printSchema()
analysis_df.show()

root
 |-- Total number of reviews: integer (nullable = true)
 |-- Number of 5-star reviews: integer (nullable = true)
 |-- Percentage of 5-star reviews (Paid) - %: float (nullable = true)
 |-- Percentage of 5-star reviews (Unpaid) - %: float (nullable = true)

+-----------------------+------------------------+---------------------------------------+-----------------------------------------+
|Total number of reviews|Number of 5-star reviews|Percentage of 5-star reviews (Paid) - %|Percentage of 5-star reviews (Unpaid) - %|
+-----------------------+------------------------+---------------------------------------+-----------------------------------------+
|                 133416|                   76294|                             0.28835818|                                 99.71164|
+-----------------------+------------------------+---------------------------------------+-----------------------------------------+

